# BertSum

## Import library

In [1]:
# # colab
# from google.colab import drive
# drive.mount('/content/drive')
# PROJECT_ID = "/Projects/202011_DACON_Summarization"

In [1]:
!pwd

/home/uoneway/Project/PreSumm_ko/notebooks


In [2]:
ROOT_DIR = "/home/uoneway/Project"

In [4]:
import numpy as np 
import pandas as pd
import os
import json

# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
# sns.set(style='white', context='notebook', palette='deep')

import warnings
warnings.filterwarnings(action='ignore')

if get_ipython():
    print("jupyter envirionment")
    from tqdm import tqdm_notebook as tqdm  #이거 안해주면 한 줄씩 출력됨 ;;
else:
    print("command shell envirionment")
    from tqdm import tqdm

jupyter envirionment


In [7]:
PROJECT_DIR = ROOT_DIR + '/PreSumm_ko'
DATA_DIR = PROJECT_DIR + '/data'
RAW_DATA_DIR = DATA_DIR + '/raw'
JSON_DATA_DIR = DATA_DIR + '/json_data'
BERT_DATA_DIR = DATA_DIR + '/bert_data' 

MODEL_DIR = PROJECT_DIR + '/models'  
LOG_DIR = PROJECT_DIR + '/logs' # logs -> for storing logs information during preprocess and finetuning
RESULT_DIR = PROJECT_DIR + '/result' 


## preprocessing

Our data process includes the following four steps:  
- 1. Convert our raw data to **.story** format and store in */kaggle/working/news_sum/* . This enables us to use **preprocess.py** code to do further process  
- 2. Tokenize **.story** file via using [**StanfordCoreNLP**](https://stanfordnlp.github.io/CoreNLP/index.html). Stanford CoreNLP is written in **Java**; recent releases require Java **1.8+**. You need to have Java installed to run CoreNLP. However, you can interact with CoreNLP via the command-line or its web service; many people use CoreNLP while writing their own code in Javascript, Python, or some other language.  

    (*Here we activate Stanford CoreNLP by the code in **preprocess.py***)
- 3. Convert tokenized data as **json** format 
- 4. Convert **json** file to **.pt** format for model finetuning


**folder name -> data file**
https://www.kaggle.com/uoneway/text-summarization-with-presumm-finetuning/edit




### Import data

In [8]:
with open(RAW_DATA_DIR + '/ext/train.jsonl', 'r') as json_file:
    json_list = list(json_file)

trains = []
for json_str in json_list:
    line = json.loads(json_str)
    trains.append(line)

train_df = pd.DataFrame(trains)
train_df['extractive_sents'] = train_df.apply(lambda row: list(np.array(row['article_original'])[row['extractive']]) , axis=1)
train_df

,media,id,article_original,abstractive,extractive,extractive_sents
0,당진시대,327827480,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]",[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...
1,국제신문,339840364,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]",[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...
2,기호일보,371071597,"[인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아, 국유지로...",16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]",[16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가...
3,대구일보,354806783,[대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다...,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]",[대구·경북첨단의료산업진흥재단(이하 대구첨복재단) 의약생산센터는 주세제 특수제제인 ...
4,대구신문,347022773,[국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 ...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]",[식품의약품안전처는 29일 여름철 어린이가 즐겨 마시는 탄산음료 282개와 혼합음료...
...,...,...,...,...,...,...
42798,대구신문,334074661,[칠곡북삼초등학교(교장 서금자)에서는 지난달 7일부터 18일까지 3-5학년 학생 중...,지난달 7일부터 18일까지 칠곡북삼초등학교는 학생들에게 실제 영어 사용에 대한 자신...,"[0, 1, 2]",[칠곡북삼초등학교(교장 서금자)에서는 지난달 7일부터 18일까지 3-5학년 학생 중...
42799,대구일보,362902654,"[김상훈 의원, 최근 5년간, LH공공임대 임차인이 사망했으나 상속되거나 반환되지 ...",김상훈 자유한국당 의원은 최근 5년간 임차인의 사망으로 반환되지 못한 3천479가구...,"[1, 2, 4]","[최근 5년간, LH공공임대 임차인이 사망했으나 상속되거나 반환되지 못한 임대보증금..."
42800,기호일보,355310553,[▲ 5일 오전 파주 임진각에서 열린 ‘DMZ 155마일 걷기’ 출정식에서 걷기대원...,경기도는 DMZ의 의미와 평화에 관한 국민들의 인식을 새롭게 하고자 'DMZ 1...,"[2, 3, 6]",[경기도의 ‘DMZ 155마일 걷기’ 행사가 5일 오전 파주 임진각에서 열린 출정식...
42801,당진시대,288844233,"[당진지역 휘발유 평균 가격은 1534원, 경유 평균가격은 1343원으로 조사됐다....",지난 14일 기준 당진지역 휘발유 중 가장 저렴한 곳의 가격은 AJ토탈(주)당진주유...,"[4, 6, 0]",[지난 14일 기준 휘발유 가격이 가장 저렴한 곳은 AJ토탈(주)당진주유소(신평)였...


### DIR 만들기

In [ ]:
import os 
os.chdir(PreSumm_DIR + '/data')

!mkdir json_data json_data/train_ext json_data/train_abs json_data/test
!mkdir bert_data bert_data/train_ext bert_data/train_abs bert_data/test
# !mkdir logs
# !mkdir temp
# !mkdir bertsumextabs 

mkdir: cannot create directory ‘json_data’: File exists
mkdir: cannot create directory ‘json_data/train_ext’: File exists
mkdir: cannot create directory ‘json_data/train_abs’: File exists
mkdir: cannot create directory ‘bert_data’: File exists
mkdir: cannot create directory ‘bert_data/train_ext’: File exists
mkdir: cannot create directory ‘bert_data/train_abs’: File exists


In [ ]:
# !mv bert_data/train.* bert_data/train_ext

In [ ]:
# !mv json_data/test.* json_data/test

### DATA 변환

##### Mecab 설치

In [15]:
%%capture

# mecab 설치
import os
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.2')
!./configure
!make
!make check
!make install
!ldconfig

# GNU M4 설치
os.chdir('/tmp/')
!wget http://ftp.gnu.org/gnu/m4/m4-1.4.9.tar.gz
!tar -zvxf m4-1.4.9.tar.gz
os.chdir('/tmp/m4-1.4.9')
!./configure
!make
!make install

# autoconf 설치
os.chdir('/tmp')
!curl -OL http://ftpmirror.gnu.org/autoconf/autoconf-2.69.tar.gz
!tar xzf autoconf-2.69.tar.gz
os.chdir('/tmp/autoconf-2.69')
!./configure --prefix=/usr/local
!make
!make install
!export PATH=/usr/local/bin

# automake 설치
os.chdir('/tmp')
!curl -LO http://ftpmirror.gnu.org/automake/automake-1.11.tar.gz
!tar -zxvf automake-1.11.tar.gz
os.chdir('/tmp/automake-1.11')
!./configure
!make
!make install

# mecab dictionary 설치
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
!tar -zxvf mecab-ko-dic-2.1.1-20180720.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.1.1-20180720')
!./autogen.sh
!./configure
!make
!make install

# mecab python 설치
os.chdir('/content')

!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')

!python3 setup.py build
!python3 setup.py install

In [11]:
!pip install kss

  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251547 sha256=1d849724354b9ac15b34d699e836b42553da87ce6d849d4eaba572fa1a17838e
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss


#### make json file

In [12]:
# `[{"src": [["(",],
#            [],
#     ],
#     "tgt": [["(",],
#             [],
#     ]
#   },
#   {}
# ]`

In [9]:
import re
import MeCab
from bs4 import BeautifulSoup
import kss

In [27]:
special_symbols_in_dict = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-']
for i in special_symbols_in_dict:
    print(i)

!
"
#
$
%
&
'
(
)
*
+
,
-


In [37]:
# special_symbols_in_dict = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-']
# unused_tags = ['SF', 'SE', 'SSO', 'SSC', 'SC', 'SY']

def korean_tokenizer(text, unused_tags=None, print_tag=False): 
    # assert if use_tags is None or unuse_tags is None
    
    tokenizer = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ko-dic")
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")]
    result = []
    
    if unused_tags:
        for word_ in word_tag[:-2]:
            word = word_.split("\t")
            tag = word[1].split(",")[0]
            if tag not in unused_tags:
                if print_tag:
                    result.append((word[0], tag))
                else:
                    result.append(word[0]) 
    else:
        for word_ in word_tag[:-2]:
            word = word_.split("\t")
            result.append(word[0]) 

    return result

def number_split(sentence):
    # 1. 공백 이후 숫자로 시작하는 경우만(문자+숫자+문자, 문자+숫자 케이스는 제외), 해당 숫자와 그 뒤 문자를 분리
    num_str_pattern = re.compile(r'(\s\d+)([^\d\s])')
    sentence = re.sub(num_str_pattern, r'\1 \2', sentence)

    # 2. 공백으로 sentence를 분리 후 숫자인경우만 공백 넣어주기
    #numbers_reg = re.compile("\s\d{2,}\s")
    sentence_fixed = ''
    for token in sentence.split():
        if token.isnumeric():
            token = ' '.join(token)
        sentence_fixed+=' '+token
    return sentence_fixed

def noise_remove(text):
    text = text.lower()
    
    # url 대체
    url_pattern = re.compile(r'https?://\S*|www\.\S*')
    text = url_pattern.sub(r'URL', text)

    # html 삭제
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    # 숫자 중간에 공백 삽입하기
    text = number_split(text)
    #number_pattern = re.compile('\w*\d\w*') 
#     number_pattern = re.compile('\d+') 
#     text = number_pattern.sub(r'[[NUMBER]]', text)
    

    # PUCTUACTION_TO_REMOVED = string.punctuation.translate(str.maketrans('', '', '\"\'#$%&\\@'))  # !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ 중 적은것을 제외한 나머지를 삭제
    # text = text.translate(str.maketrans(PUCTUACTION_TO_REMOVED, ' '*len(PUCTUACTION_TO_REMOVED))) 

    # remove_redundant_white_spaces
    text = re.sub(' +', ' ', text)
    
    return text

def preprocessing(text, tokenizer=None, unused_tags=None):
    text = noise_remove(text)
    if tokenizer is not None:
        text = tokenizer(text, unused_tags=unused_tags)
        text = ' '.join(text)

    return text
# korean_tokenizer
text = "■위치 : 당진시 무수동 7길 123 (하늘채코오롱아파트 정문 앞) ■문의 : 041-355-3282! http://dsf/com <a> dsf <a/>"
preprocessing(text, korean_tokenizer, unused_tags=unused_tags)


'위치 당진시 무수동 7 길 1 2 3 하늘 채 코오롱 아파트 정문 앞 문의 0 4 1 355 3282 URL dsf'

In [29]:
for i, row in train_df.iloc[0:100].iterrows():
            print(len(row['article_original']))

9
21
19
10
13
5
7
9
8
14
67
17
22
15
13
16
9
11
13
13
19
11
9
9
7
13
10
15
13
8
10
12
11
13
17
8
10
17
16
20
13
8
7
12
8
8
17
10
24
10
8
11
10
12
9
14
9
8
10
10
8
13
8
9
25
10
19
9
14
13
11
17
7
10
24
10
16
10
17
22
13
29
10
15
22
12
8
10
11
11
5
8
9
7
17
11
11
12
13
22


In [ ]:
import json

def create_json_files(df, path='', type=None, summary_target = 'abs'):
    NUM_DOCS_IN_ONE_FILE = 1000
    start_idx_list = list(range(0, len(df), NUM_DOCS_IN_ONE_FILE))

    for start_idx in tqdm(start_idx_list):
        end_idx = start_idx + NUM_DOCS_IN_ONE_FILE
        if end_idx > len(df):
            end_idx = len(df)  # -1로 하니 안됨...

        #정렬을 위해 앞에 0 채워주기
        length = len(str(len(test_df)))
        start_idx_str = (length - len(str(start_idx)))*'0' + str(start_idx)
        end_idx_str = (length - len(str(end_idx-1)))*'0' + str(end_idx-1)

        file_name = os.path.join(path,f"{type}.{start_idx_str}_{end_idx_str}.json")
        
        json_list = []
        for _, row in df.iloc[start_idx:end_idx].iterrows():
            original_sents_list = [preprocessing(original_sent, korean_tokenizer).split()
                                    for original_sent in row['article_original']]

            if summary_target == 'ext':
                summary_sents_list = [preprocessing(original_sent, korean_tokenizer).split()
                                    for original_sent in row['extractive_sents']] if type == 'train' else []
            else:
                summary_sents_list = []
                sents_splited = kss.split_sentences(row['abstractive'])
                if len(sents_splited) == 1:
                    summary_sents_list = sents_splited
                elif len(sents_splited) > 1:
                    is_multi_sent = False
                    for idx in range(len(sents_splited)):
                        if sent_splited[-1] in ['.','?' ]:
                            is_multi_sent = True
                        else:
                            is_multi_sent = False
                            break
                    if is_multi_sent:
                        summary_sents_list.append(sents_splited)
                else:
                    print("error")
                    
            
            json_list.append({'src': original_sents_list,
                              'tgt': summary_sents_list
            })
        # print(json_list)


        json_string = json.dumps(json_list, indent=4, ensure_ascii=False)
        #print(json_string)
        with open(file_name, 'w') as json_file:
            json_file.write(json_string)


In [ ]:
summary_sents_list = []
for abstractive in train_df['abstractive'].values:
    sents_splited = kss.split_sentences(abstractive)
    
    if len(sents_splited) == 1:
        # .이나 ?가 있는데도 kss가 분리하지 않은 문장들을 혹시나해서 살펴보니
        # 대부분 쉼표나 가운데점 대신 .을 사용하거나 "" 사이 인용문구 안에 들어가있는 점들. -> 괜찮.
        # aa = sents_splited[0].split('. ')
        # if len(aa) > 1:
        #     print(sents_splited)
        summary_sents_list = sents_splited
    elif len(sents_splited) > 1:
        is_multi_sent = False
        summary_sents_list = []
        for idx in range(len(sents_splited) - 1):
            # 두 문장 사이에 .이나 ?가 있는 경우는 그냥 붙여주기
            if sents_splited[idx][-1] in ['.','?' ] and idx != (len(sents_splited) - 1):
                is_multi_sent = True
            else:
                is_multi_sent = False
                break
        if is_multi_sent:
            summary_sents_list.append(sents_splited)

['평가자님, 이런 기사는 어떻게 요약하면 좋을까요?? ㅠㅠ']
["그룹 신화의 에릭이 인스타그램에 영화 '라라랜드'의 촬영배경지였던 미국 허모사비치를 배경으로 '현지에서 먹힐까? 미국편' 단체복을 입은 사진과 함께 #에릭고슬링 해시태그를 덧붙이며 센스있게 게재했다."]
['이연복 셰프를 중심으로 미국 현지에서 푸드트럭 장사에 도전하는 tvN 프로그램 \'현지에서 먹힐까? 미국편\'이 18일 밤11시 첫 방송을 앞두고 있는 가운데, 제작진이 "이연복의 짜장면이 미국에서 통할까?" 등 세 가지 관전 포인트를 공개했다.']
['현대기아차는 ‘왓카 어워드(What Car? Awards)’에서 뛰어난 1회 충전 시 주행 가능 거리와 동급 최고 수준의 공간 활용성 등 최상의 상품성과 합리적인 가격으로 높은 점수를 받아 ‘올해의 차’를 비롯해 9개 부문에서 수상했다.']
["엄마들이 모여 만든 미국 단체 '마더 컴퍼니'가 분노를 스스로 다스리는 법을 전하는 '화를 낼까? 화를 풀까?'라는 감정인지 교육용 동화를 출판하였다."]
["25일 방송된 tvN '현지에서 먹힐까? 미국편'에서는 가수 존박이 최근 열린 제작발표회를 통해 가장 인상 깊었고 다시는 만들기 싫은 음식으로 이연복 셰프의 만두로 불리는 '복만두'를 꼽았다."]


In [ ]:
create_json_files(train_df, path=JSON_DATA_DIR + '/train_abs', type='train')

KeyboardInterrupt: ignored

In [ ]:
create_json_files(test_df, path=JSON_DATA_DIR, type='test')

#### 4. Make .json file as .pt file for BERT model 

In [ ]:
!pip install -r $PreSumm_DIR/requirements.txt
!pip install transformers

In [ ]:
import os
os.chdir(PreSumm_DIR + '/src/')

# 동일한 파일명 존재하면 덮어쓰는게 아니라 넘어감
!python preprocess.py -mode format_to_bert \
                        -raw_path $JSON_DATA_DIR/train_abs -save_path $BERT_DATA_DIR/train_abs  -lower -n_cpus 1 -log_file $LOG_FILE \
                        -dataset train

Traceback (most recent call last):
  File "preprocess.py", line 8, in <module>
    from prepro import data_builder
  File "/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/src/prepro/data_builder.py", line 18, in <module>
    from others.tokenization_kobert import KoBertTokenizer
  File "/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/src/others/tokenization_kobert.py", line 23, in <module>
    from transformers import PreTrainedTokenizer
ModuleNotFoundError: No module named 'transformers'


In [ ]:
import os
os.chdir(PreSumm_DIR + '/src/')

# 동일한 파일명 존재하면 덮어쓰는게 아니라 넘어감
!python preprocess.py -mode format_to_bert \
                        -raw_path $JSON_DATA_DIR -save_path $BERT_DATA_DIR  -lower -n_cpus 1 -log_file $LOG_FILE \
                        -dataset test

2020-11-30 05:25:04.736700: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
[('test', '/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/json_data/test.0000_0099.json', Namespace(dataset='test', log_file='/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/logs/ext_bert.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/json_data', save_path='/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/bert_data', select_mode='greedy', shard_size=2000, use_bert_basic_tokenizer=False), '/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/be

#### (이전) sum -> .story data -> tokenized -> .json file


In [ ]:
# original_sents_list_sr = train_df['article_original']
# abstractive_sents_sr = train_df['abstractive']
# extractive_sents_list_sr = train_df['extractive_sents']

`temp_ext.raw_src`
- insert [CLS] [SEP] as your sentence boundaries.
- example   
this Terry Jones had a love of the absurd that contributed much to the anarchic humour of Monty Python's Flying Circus. [CLS] [SEP] His style of visual comedy, leavened with a touch of the surreal, inspired many comedians who followed him. 
(CNN) An Iranian chess referee says she is frightened to return home after she was criticized online for not wearing the appropriate headscarf during an international tournament. [CLS] [SEP] CNN contacted the Iranian Chess Federation on Tuesday but has yet to receive a response.


`temp.raw_tgt` 
- 순서에 맞춰서 한 줄씩

In [ ]:
# original_docs = ''

# for original_sents_list in original_sents_list_sr:
#     original_doc = ''
#     for idx, original_sent in enumerate(original_sents_list):
#         original_sent = str.strip(original_sent)
#         if idx == len(original_sents_list)-1:
#             original_doc += original_sent + '\n'
#         else:
#             original_doc += original_sent + ' [CLS] [SEP] ' 
#     #print(original_doc)
#     original_docs += original_doc
# #original_docs


# with open(PreSumm_DIR + '/raw_data/temp.raw_tgt', 'w') as file:
#     file.write(original_docs)

In [ ]:
# abstractive_docs = ''
# for abstractive_sents in abstractive_sents_sr:
#     abstractive_docs += abstractive_sents + '\n'

# with open(PreSumm_DIR + '/raw_data/temp_ext.raw_src', 'w') as file:
#     file.write(abstractive_docs)

In [ ]:
def joiner(sents_list, sent_type='origin'):
    if sent_type == 'origin':
        return '\n\n'.join(sents_list)
    else:  # summary
        summary = ''
        for sent in sents_list:
            summary += '\n\n@highlight\n\n' + sent
        return summary
joiner(train_df['extractive_sents'][1], 'summary')

NameError: ignored

In [ ]:

def create_story_files(df,num_file=None,filename=None):
    if num_file == None:
        num_file = len(df)
    for i in tqdm(range(num_file)):
        doc = joiner(train_df['article_original'][i]) \
            +  joiner(train_df['extractive_sents'][i], 'summary')

        file_name = os.path.join(filename,(str(i) + '.story'))
        with open(file_name,'w') as story_file:
            story_file.write(doc)

In [ ]:
# create_story_files(train_df,filename=STORY_DIR)

In [ ]:
# Download Stanford CoreNLP
# We will need Stanford CoreNLP to tokenize the data. Download it here and unzip it. Then add the following command to your bash_profile:

!export CLASSPATH=$PreSumm_DATA_DIR/stanford-corenlp-4.2.0/stanford-corenlp-4.2.0.jar

In [ ]:
import os
os.chdir(PreSumm_DIR + '/src/')

!python preprocess.py -mode tokenize -raw_path $STORY_DIR -save_path $MERGE_STORY_DIR  -log_file $LOG_FILE

Preparing to tokenize /content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/raw_stories to /content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/merged_stories_tokenized...
Making list of files to tokenize...
Tokenizing 42803 files in /content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/raw_stories and saving in /content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/merged_stories_tokenized...
Error: Could not find or load main class edu.stanford.nlp.pipeline.StanfordCoreNLP
Caused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP
Stanford CoreNLP Tokenizer has finished.
Traceback (most recent call last):
  File "preprocess.py", line 73, in <module>
    eval('data_builder.'+args.mode + '(args)')
  File "<string>", line 1, in <module>
  File "/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/src/prepro/data_builder.py", 

In [ ]:
# 3. Make tokenized data files as .json file
!python preprocess.py -mode format_to_lines -raw_path $STORY_DIR -save_path $JSON_DATA_DIR -n_cpus 1 -use_bert_basic_tokenizer false -log_file $LOG_FILE


Traceback (most recent call last):
  File "preprocess.py", line 73, in <module>
    eval('data_builder.'+args.mode + '(args)')
  File "<string>", line 1, in <module>
  File "/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/src/prepro/data_builder.py", line 335, in format_to_lines
    for line in open(pjoin(args.map_path, 'mapping_' + corpus_type + '.txt')):
FileNotFoundError: [Errno 2] No such file or directory: '../../data/mapping_valid.txt'
